<a href="https://colab.research.google.com/github/ami-iit/ADAM/blob/ik-example/examples/ik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Mujoco, adam, and mediapy.

Download also mujoco-menagerie for the panda model and the urdf needed from adam.
Set some stuff for the visualization.

In [1]:
!pip install mujoco
!pip install adam-robotics
!pip install -q mediapy
!git clone https://github.com/google-deepmind/mujoco_menagerie.git
!wget https://raw.githubusercontent.com/bulletphysics/bullet3/master/examples/pybullet/gym/pybullet_data/franka_panda/panda.urdf
# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 MB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for urdf-parser-py: filename=urdf_parser_py-0.0.4-py3-none-any.whl size=14549 sha256=1938c2d5d4910bf86c6f92c4763a19aabb3b26b203b787138faddf01346f5ec6
  Stored in directory: /root/.cache/pip/wheels/09/0d/9f/f0e2fb2ae580623b840d041d4aab0a9df02f5047f283f4ecd1
Successfully built urdf-parser-py
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.8 MB/s eta 0:00:00
Cloning into 'mujoco_menagerie'...
remote: Enumerating objects: 1433, done.
remote: Counting objects: 100% (265/265), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 1433 (delta 150), reused 112 (delta 104), pack-reused 1168
Re

## Import packages

In [2]:
import mujoco
import mediapy as media
from adam.casadi import KinDynComputations
import numpy as np
import casadi as cs


## Import the panda scene in mujoco

In [3]:
# load scene from xml
model = mujoco.MjModel.from_xml_path("mujoco_menagerie/franka_emika_panda/scene.xml")


## Import urdf in adam

Set the commanded joint list and impor the urdf in adam.

For now I have to use a separate urdf for adam.
An importer for a mujoco model could be an idea for the future!

In [4]:
joints_name_list = ['panda_joint1', 'panda_joint2', 'panda_joint3', 'panda_joint4', 'panda_joint5', 'panda_joint6', 'panda_joint7', 'panda_joint8']

kindyn = KinDynComputations(urdfstring="panda.urdf", joints_name_list=joints_name_list)


Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link0']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link1']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link2']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link4']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link5']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link6']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link7']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_hand']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_leftfinger']/collision[1]
Unknown tag "contact" in /robot[@name='panda']/link[@name='panda_leftfinger']
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_rightfinger']/collision[1]
Unknown tag "contact" in /robot[@name='pan

## A wrapper interface with mujoco

In [5]:
class MujocoWrapper:
  # a simple wrapper to use mujoco as a simulator
  def __init__(self, model, joints_list=None):
    self.model = model
    self.data = mujoco.MjData(model)
    self.renderer = mujoco.Renderer(self.model)

  def set_qpos(self, qpos):
    # set the joint positions
    self.data.qpos[:] = qpos
    mujoco.mj_forward(self.model, self.data)

  def get_qpos(self):
    # get the joint positions
    return self.data.qpos[:]

  def render(self):
    # render the scene and return the frame
    mujoco.mj_forward(self.model, self.data)
    self.renderer.update_scene(self.data)
    return self.renderer.render()

  def step(self):
    # step the simulation
    mujoco.mj_step(self.model, self.data)

  def set_qvel(self, qvel):
    # set the joint velocities
    self.data.qvel[:] = qvel
    mujoco.mj_forward(self.model, self.data)

wrapper = MujocoWrapper(model)


# Model IK

A simple IK with damped least squares and manipulability regularization.

In [8]:
# setup inverse kinematics


w_H_ee = kindyn.forward_kinematics_fun("panda_hand")
J = kindyn.jacobian_fun("panda_hand")

class InverseKinematics:
    def __init__(self, w_H_ee, J):
        self.q_cs = cs.SX.sym("joint_positions", 8)
        self.q_dot_cs = cs.SX.sym("joint_velocities", 8)
        w_H_b = np.eye(4)
        self.w_p_ee = cs.Function("w_H_ee", [self.q_cs], [w_H_ee(w_H_b, self.q_cs)[:3, 3]])
        self.J = cs.Function("J", [self.q_cs], [J(w_H_b, self.q_cs)[:3, 6:]])
        manipulability = cs.sqrt(cs.det(self.J(self.q_cs) @ self.J(self.q_cs).T))
        # use casadi tools to compute the gradient of the manipulability
        q_dot_manipulability = cs.jacobian(manipulability, self.q_cs).T
        self.q_dot_manipulability = cs.Function("q_dot_manipulability", [self.q_cs], [q_dot_manipulability])

    def __call__(self, q, w_p_ee_desired):
        w_p_ee = self.w_p_ee(q)
        ee_error =  w_p_ee_desired - w_p_ee
        J = self.J(q)
        K_p = 2
        N = self.null_space_projection(J)
        q_dot_bias = self.q_dot_manipulability(q)
        # damped least squares
        damping_factor = 1e-2
        damped_pinv = np.linalg.inv(J.T @ J + damping_factor * np.eye(8)) @ J.T
        q_dot = damped_pinv @ (K_p * ee_error) + N @ q_dot_bias
        return np.array(q_dot).squeeze()

    def null_space_projection(self, J):
        return np.eye(8) - np.linalg.pinv(J) @ J

ik = InverseKinematics(w_H_ee, J)


# Simulation loop

We set the joint velocities as control input in Mujoco.
We retrieve the joint positions as measurement from Mujoco and set them as feedback for the IK.


In [ ]:
# start mujoco simulation along with control
duration = 10  # (seconds)
framerate = 100  # (Hz)

# Simulate and display video.

des_ee_pos_numeric = np.array([0.4, 1.0, 0.4])

frames = []
mujoco.mj_resetData(wrapper.model, wrapper.data)
i = 0
while wrapper.data.time < duration:
  wrapper.step()
  if len(frames) < wrapper.data.time * framerate:
    i += 1
    q0_numeric = wrapper.get_qpos()
    # remove the last joint since they are not controlled
    q0_numeric = q0_numeric[:-1]
    # set the initial condition
    sol_q_dot = ik(q0_numeric, des_ee_pos_numeric)
    sol_q_dot = np.concatenate((sol_q_dot, np.zeros(1)))
    wrapper.set_qvel(sol_q_dot)
    pixels = wrapper.render()
    frames.append(pixels)
  if wrapper.data.time > 2:
    # change the desired ee position
    des_ee_pos_numeric = np.array([2.0, 0.0, 0.2])
    sol_q_dot = ik(q0_numeric, des_ee_pos_numeric)
  if wrapper.data.time > 4:
    # change the desired ee position
    des_ee_pos_numeric = np.array([0.0, 0.5, 0.4])
    sol_q_dot = ik(q0_numeric, des_ee_pos_numeric)
  if wrapper.data.time > 6:
    # change the desired ee position
    des_ee_pos_numeric = np.array([0.0, -0.6, 0.6])
    sol_q_dot = ik(q0_numeric, des_ee_pos_numeric)

media.show_video(frames, fps=framerate)
